# Notebook pour transformer des fichiers excel en modeles django


In [1]:
!pip install jr_data_science

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import jr_data_science as ds
from jr_data_science import functions_de_stats_sur_df
from jr_data_science import useful_functions
from jr_data_science import functions_de_decouverte_de_fichiers

In [3]:
import pandas as pd

In [4]:
def description(df):
    sep = "|"
    for col in df.columns:
        print (sep.join([str(x) for x in [col, df[col].nunique(), 1 - df[col].isnull().sum()/df.shape[0]]]))

In [5]:

def get_connection(path_sqlite3):
    import sqlite3
    conn = sqlite3.connect(path_sqlite3)
    return conn

def excel_to_sqlite(path_xls, path_sqlite3, table_name, if_exists="replace"):
    import pandas as pd
    df = pd.read_excel(path_xls)
    df_to_sqlite(df, path_sqlite3, table_name, if_exists)
        
def df_to_sqlite(df, path_sqlite3, table_name, if_exists="replace"):
    conn = get_connection(path_sqlite3)
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)

In [6]:
from jr_data_science import functions_pour_analyser_des_df

In [7]:
def clean_string(string):
    string = string.replace("é", "e")
    string = string.replace("è", "e")
    string = string.replace(" ", "_")
    string = string.replace("€", "_eur")    

    a = "".join([x.lower()  for x in string if x.lower() in "_azertyuiopqsdfghjklmwxcvbn1234567890".lower()])
    a = a.replace("__", "_").replace("__", "_")
    a = a[:-1] if a.endswith("_") else a
    return a

In [8]:
def  write_df_to_django_model(output_file, df, nom_df="nom_df"):
    create_dir_if_not_exit(output_file)
    with open(output_file, "w") as f:
        f.writelines(df_to_django_model(df, nom_df))

In [53]:
    conversion = {  "datetime64[ns]" : "models.DateTimeField(blank=False, null=True)",
                    "int64"          : "models.IntegerField(blank=True, null=True)"   ,
                    "object"         : "models.TextField(blank=True, null=True)"      ,
                    "float64"        : "models.FloatField(blank=True, null=True)"     }


In [56]:
conversion.get("coucou", conversion["object"])

'models.TextField(blank=True, null=True)'

In [66]:
def df_to_django_model(df, nom_df="nom_df", debug=False):
    """
    Output on screen a proposal of a django model.
    """
    conversion = {  "datetime64[ns]" : "models.DateTimeField(blank=False, null=True)",
                    "int64"          : "models.IntegerField(blank=True, null=True)"   ,
                    "object"         : "models.TextField(blank=True, null=True)"      ,
                    "float64"        : "models.FloatField(blank=True, null=True)"     }
    max_len = max([len(x) for x in df.columns])
    nb_car  = max_len + 2
    tab      = " "*4
    to_print ="""
from django.db 						import models
from django.contrib.auth.models 	import User
from django.forms 					import ModelForm
from django.urls import reverse
"""
    to_print +="class %s (models.Model):\n"%nom_df
    primary_key = "primary_key=True"
    for nom_col, dtype in zip(df.columns, df.dtypes):
        blank     = nb_car - len(nom_col)
        type_col  = conversion.get(dtype, conversion["object"])
        to_print +="{tab}{nom_col} {blank}= {type_col}{primary_key}\n".format(tab=tab, nom_col=nom_col, blank=" "*blank, type_col=type_col, primary_key=primary_key)
        primary_key=""
    if debug : print(to_print)
        
    to_print +="{tab}def __str__(self):\n".format(tab=tab)
    to_print +="{tab}{tab}r = ''\n".format(tab=tab)
    for nom_col  in df.columns[:3]:
        to_print +="{tab}{tab}r+= str(self.{nom_col})\n".format(tab=tab, nom_col=nom_col)
    to_print +="{tab}{tab}return r".format(tab=tab)
    
    return to_print


In [10]:
def write_auto_form(output_file, nom_model, df):
    create_dir_if_not_exit(output_file)
    with open(output_file, "w") as f:
        f.writelines(df_to_auto_form(nom_model, df))

In [11]:
def df_to_auto_form(nom_model, df, debug=False):
    to_print =""
    fields = "                   ".join( ["""  "%s",\n"""%x for x in df.columns] )
    to_print = """    
from .models import *
from django.db import models
from django.forms import ModelForm

class {model}Form(ModelForm):
    error_css_class     = 'error'
    required_css_class  = 'required'
    class Meta:
        model   = {model}
        fields = [ {fields} ]
                   
    def __init__(self, *args, **kwargs):
        super({model}Form, self).__init__(*args, **kwargs)  # Call to ModelForm constructor
        height  = 1
        width   = 40
        for to_change in self.fields:
            self.fields[to_change ].widget.attrs['cols'] = width
            self.fields[to_change ].widget.attrs['rows'] = height


""".format(model= nom_model,  fields=fields)
    if debug : print(to_print)
    for action in ["Create", "Detail", "Update", "Delete", "List"]:
        to_print += """
class {action}{model}Form({model}Form):
    def __init__(self, *args, **kwargs):
        super({action}{model}Form, self).__init__(*args, **kwargs)  # Call to ModelForm constructor
    """.format(model= nom_model, action=action, fields=fields)
    if debug : print(to_print)
    return to_print

In [12]:
def write_auto_views(output_dir, nom_model):
    """
    Ecrit un fichier de vue automatiques pour django.
    """
    import os
    create_dir_if_not_exit(os.path.join(output_dir, "touch"))
    to_write = df_to_auto_views(nom_model)
    with open(os.path.join(output_dir, "views.py"), "w") as f:
        f.writelines(to_write)

In [13]:
def df_to_auto_views(nom_model, debug=False):
    to_print = """
from django.views.generic           import ListView
from django.views.generic           import UpdateView
from django.views.generic           import FormView
from django.views.generic           import DetailView
from django.views.generic           import DeleteView
from django.views.generic           import CreateView
from django.urls                    import reverse_lazy
from django.contrib.auth.decorators import login_required
from django.utils.decorators        import method_decorator
from .models import *
from .forms import *

    """
    for action in ["Create", "Detail", "Update", "Delete", "List"]:
        to_print += """
class {model}{action}({action}View):
    model                = {model}
    form_class           = {action}{model}Form
    template_name_suffix = '_{action_small}'
    """.format(model= nom_model, action=action, action_small=action.lower())
        if action in ["Delete"]:
            to_print +="""
    success_url          = reverse_lazy('List{model}')
    """.format(model=nom_model)
    if debug : print(to_print)
    return to_print

In [14]:
def create_dir_if_not_exit(filepath):
    import os
    os.makedirs(os.path.dirname(filepath), exist_ok=True)


In [15]:
def write_url(output_path, model):
    """
    Ecrit les urls django dans un fichier output (call creer urls)
    
    output_path : chemin complet vers le fichier cible
    Return :
        Void.
    """
    create_dir_if_not_exit(output_path)
    with open(output_path, "w") as f:
        f.writelines(creer_urls(model))
        

In [16]:
def creer_urls(model, debug=False):
    """
    Créer une chaine de caractere avec les urls django pour les vues auto d'une app.
    
    Return : 
        string
    """
    to_write = """
    
from django.urls import path
from django.contrib.auth.decorators import login_required
from django.urls import path

from .views import *
from django.contrib.auth.decorators import login_required
from django.urls import path

urlpatterns = [ path('Create{app}/'            , login_required({app}Create.as_view())   ,  name="Creer{app}"    ),
                 path('List{app}/'              , login_required({app}List.as_view())     , name="List{app}"),
                 path('Update{app}/<int:pk>'    , login_required({app}Update.as_view())   , name="Update{app}"   ),
                 path('Detail{app}/<int:pk>'    , login_required({app}Detail.as_view())   , name="Detail{app}"   ),
                 path('Delete{app}/<int:pk>'    , login_required({app}Delete.as_view())   , name="Delete{app}"   ),]

""".format(app=model)
    if debug : print(to_write)
    return to_write


In [17]:
def get_upper_dir(dirpath):
    import os
    if dirpath.endswith("/"):
        dirpath = dirpath[:-1]        
    upper_dir = os.path.dirname(dirpath)
    return upper_dir


In [18]:
def get_pk_from_template_dir(template_dir):
    import os
    app_dir = get_upper_dir(get_upper_dir(template_dir))
    model_path = os.path.join(app_dir, "models.py")
    return get_pk_from_modelfile(model_path)

In [19]:
def get_pk_from_modelfile(model_file_path):
    with open(model_file_path, "r") as f:
        r = [ x for x in f.readlines() if  "primary_key" in x ]
    return r[0].split()[0]
    

In [20]:
def create_template(template_dir, name_model, debug=False):

    urls = {
        "Create" : "Creer" , 
        "List"  : "List", 
        "Update" : "Update", 
        "Detail"  : "Detail", 
        "Delete" : "Delete", 
        "confirm_delete" : "", 
        "Form" : "Creer"
    }
    pk0 = str(get_pk_from_template_dir(template_dir))
    
    for action in ["Create", "List", "Update", "Detail", "Delete", "confirm_delete", "Form"]:
        path_ = template_dir  + name_model + "_"+  action + ".html"
        path_ = path_.lower()
        create_dir_if_not_exit(path_)
        bof = urls[action]
        pk = "pk=object.%s"%pk0 if bof not in ["Creer", "List"] else ""
        url = bof+name_model
        if debug : print("Template into %s"%path_)
        if bof in ["Creer"]:
            write_template_creer(path_, url, pk)
        elif bof in ["List"]:
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_list(path_, url, pk)
        elif bof in ["Update"]:
            url = "Delete"+name_model
            pk = "pk=object.%s"%pk0
            write_template_update(path_, url, pk)
        elif bof in ["Detail"]:        
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_detail(path_, url, pk)
        elif bof in ["Delete"]:    
            url = "Update"+name_model
            pk = "pk=object.%s"%pk0
            write_template_delete(path_, url, pk)
        else:
            write_template_display(path_, url, pk)
            


        

In [21]:
def write_template_delete(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
    
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<form method="post">{% csrf_token %}
    <p>Are you sure you want to delete "{{ object }}"?</p>
    <input type="submit" value="Confirm" />
</form>
        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a> <br>
        {% endblock %}
                """
            f.writelines(to_write)

In [22]:
def write_template_list(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
{% extends "base.html" %}
{% block content %}
    <div widt="70%">
         {% for object in object_list %}
            
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a> <br>
         {% endfor %}        
    </div>


{% endblock %}
        """
            f.writelines(to_write)

In [23]:
def write_template_creer(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<table>
        <th colspan="2">
        <h4>Créer </h4>
</th>
        <form method="post">{% csrf_token %}
            {{form.as_table}}
            <tr>
            <td>
</td>
<td>
            <input type="submit" value="Créer" class="btn btn-primary" />
</td>
</tr>
</table>
        {% endblock %}
        """
            f.writelines(to_write)

In [24]:
def write_template_update(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
<table>
        <th colspan="2">
        <h4>Mise à jour </h4>
</th>
        <form method="post">{% csrf_token %}
            {{form.as_table}}
            <tr>
            <td>
</td>
<td>
            <input type="submit" value="Mettre à jour" class="btn btn-primary" />
            </form>
</td>
</tr>
       <tr>
            <td>
</td>
<td>
            <a href={% url '"""+ url + """' """ + pk + """ %}> Delete </a>
</td>
</tr>
</table>
        {% endblock %}
        """
            f.writelines(to_write)

In [25]:
def write_template_unique(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
        <table>
            
                {{object}}
            <tr>
                <td>
                </td>
                <td>
                </td>
            </tr>
        </table>
        </div>
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} > Modifier {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [26]:
def write_template_detail(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">

             {{object}}<br>

        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >Update {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [27]:
def write_template_display(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """    
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">

             {{object}}<br>

        </div>

            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} >{{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [28]:
def write_template_unique(path_, url, pk):
    """
    Ecrit un fichier de template html.
    """
    with open(path_, "w", encoding='utf-8') as f: 
            to_write = u"""
        {% extends "base.html" %}
        {% block content %}
        <div widt="70%">
        <table>
            
                {{object}}
            <tr>
                <td>
                </td>
                <td>
                </td>
            </tr>
        </table>
        </div>
            <a href={%  url '""" + "%s"%(url) + """' """+ pk +""" %} > Modifier {{object}} </a>
        {% endblock %}
        """
            f.writelines(to_write)

In [29]:
def write_to_csv(csv_path, df, debug=False):
    create_dir_if_not_exit(csv_path)    
    if debug : print("going to write to {path}".format(path=csv_path))
    df.to_csv(csv_path, encoding="UTF-8", index=False, sep="|")

In [30]:
def get_connection(path_sqlite3):
    import sqlite3
    conn = sqlite3.connect(path_sqlite3)
    return conn
def get_df_from_sqlite_table(path_sqlite3, table_name):
    import pandas as pd
    cnx = get_connection(path_sqlite3)
    df = pd.read_sql_query("SELECT * FROM %s"%table_name, cnx)
    return df

def excel_to_sqlite(path_xls, path_sqlite3, table_name, if_exists="replace"):
    import pandas as pd
    df = pd.read_excel(path_xls)
    df_to_sqlite(df, path_sqlite3, table_name, if_exists, index=False)
        
def df_to_sqlite(df, path_sqlite3, table_name, if_exists="replace"):
    conn = get_connection(path_sqlite3)
    df.to_sql(table_name, conn, if_exists=if_exists, index=False)

In [37]:
def lister_les_urls(app):
    actions = ["Creer","List"]#,"Update","Detail","Delete",]
    print("<tr><td>{app}</br>".format(app=app))
    for action in actions:
        print("<a href={{% url '{action}{app}' %}}>{action} {app}</a>".format(action=action, app=app))
    print("</td></tr>")

        


# USAGE

In [42]:
import os

In [68]:
!pwd

/Users/romain/PycharmProjects/CCVS/zz_non_app/notebooks


In [78]:
base_dir = "/Users/romain/OneDrive/Documents/__Memorandum/_____encours/prospect/2018/2018065xx_ayming/AdvancedSolutions/app/techdata"
df_path = "qmt.df"
if base_dir.endswith("/"):
    base_dir = base_dir[:-1]
df_path = os.path.join(base_dir, df_path)

In [79]:
df = pd.read_pickle(df_path)
df.columns = [clean_string(x) for x in df.columns]
nom        = functions_de_decouverte_de_fichiers.get_file_name(df_path)
model      = nom[:nom.find(".")]
nom_app    = model
lister_les_urls(model)
write_df_to_django_model("{base_dir}/{nom_app}/models.py".format(base_dir=base_dir, nom_app=nom_app), df, nom_app)
write_auto_views("{base_dir}/{nom_app}".format(base_dir=base_dir, nom_app=nom_app), model)
create_template("{base_dir}/{nom_app}/templates/{nom_app}/".format(base_dir=base_dir, nom_app=nom_app), model)
write_url("{base_dir}/{nom_app}/urls.py".format(base_dir=base_dir, nom_app=nom_app), model)    
write_auto_form("{base_dir}/{nom_app}/forms.py".format(base_dir=base_dir, nom_app=nom_app),model, df)
write_to_csv("{base_dir}/{nom_app}/data/{nom_app}.csv".format(base_dir=base_dir, nom_app=nom_app), df, True)

with open(os.path.join(base_dir, os.path.join(os.path.basename(base_dir), "urls.py")), "a") as f:
    f.write("\nurlpatterns += [path('{nom_app}/', include('{nom_app}.urls'),)]".format(nom_app=nom_app))
with open(os.path.join(base_dir, os.path.join(os.path.basename(base_dir), "settings.py")), "a") as f:
    f.write("\nINSTALLED_APPS += [ '{nom_app}']".format(nom_app=nom_app))


<tr><td>qmt</br>
<a href={% url 'Creerqmt' %}>Creer qmt</a>
<a href={% url 'Listqmt' %}>List qmt</a>
</td></tr>
going to write to /Users/romain/OneDrive/Documents/__Memorandum/_____encours/prospect/2018/2018065xx_ayming/AdvancedSolutions/app/techdata/qmt/data/qmt.csv


# Une fois ici, il faut définir les contraintes sur les tables dans les fichiers models.py des app
# ensuite pour insérer les df dans les tables, il faut ajouter les mots "id" à la fin des colonnes des champs qui ont été référencés comme des foreignkey dans les models
# c'est cette dernière étape qu'on essaie ici

In [38]:
paths = [
"./../../zz_non_app/data/pour_romain/adherents/Cotisations.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Fichier_des_adherents.xlsx",
"./../../zz_non_app/data/pour_romain/sommaires_revue/Auteurs_details.xlsx",

"./../../zz_non_app/data/pour_romain/sommaires_revue/Revues.xlsx",

"./../../zz_non_app/data/pour_romain/sommaires_revue/Details_des_articles.xlsx",


"./../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnes_HetPl.xlsx",
"./../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnements_Renouvellements_HetPl.xlsx",
"./../../zz_non_app/data/pour_romain/homme_et_plantes/Fournisseurs_d_abonnements.xlsx",

"./../../zz_non_app/data/pour_romain/adherents/Fiches_evenements.xlsx",
"./../../zz_non_app/data/pour_romain/sommaires_revue/Ecriture.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Cotisations_des_adherents.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Situation_de_la_collection.xlsx",
"./../../zz_non_app/data/pour_romain/adherents/Collections_Archives_mai_2006.xlsx",
]

In [39]:
paths = ["./../../zz_non_app/data/pour_romain/homme_et_plantes/Abonnements_Renouvellements_HetPl.xlsx",]

In [40]:
for path in paths:
    nom        = functions_de_decouverte_de_fichiers.get_file_name(path)
    df         = pd.read_excel(path)
    df.columns = [clean_string(x) for x in df.columns]
    model      = nom[:nom.find(".")]
    nom_app    = model
    table_name="{appname}2_{appname_lower}2".format(appname=nom_app,appname_lower=nom_app.lower())
    print(table_name, df.shape)
    #df_to_sqlite(df, path_sqlite3, table_name)
    


Abonnements_Renouvellements_HetPl2_abonnements_renouvellements_hetpl2 (16845, 11)


In [41]:
df

,n_abonnement,n_abonne,reglement,trimestre,date_denregistrement,commentaires,hetp_1er_n_servir,duree_abonnement,dernier_n_servir,reference_commande,date_de_modification
0,87,766.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11069,NaN
1,88,767.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11070,NaN
2,89,768.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11071,NaN
3,90,769.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11072,NaN
4,91,770.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11073,NaN
5,92,771.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11076,NaN
6,93,772.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11078,NaN
7,94,773.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11079,NaN
8,95,774.0,76,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11080,NaN
9,96,775.0,60,1er trimestre 02,2002-02-11,2 ans,41.0,2.0,48.0,11082,NaN


In [42]:
keys = [ "n_abonne", ]

In [43]:
df = pd.read_excel(path)
df.columns = [clean_string(x) for x in df.columns]
df.columns = ["%s_id"%x if x in keys else x for x in df.columns ]
df.head(2).T

,0,1
n_abonnement,87,88
n_abonne_id,766,767
reglement,60,60
trimestre,1er trimestre 02,1er trimestre 02
date_denregistrement,2002-02-11 00:00:00,2002-02-11 00:00:00
commentaires,2 ans,2 ans
hetp_1er_n_servir,41,41
duree_abonnement,2,2
dernier_n_servir,48,48
reference_commande,11069,11070


In [44]:
import os

In [46]:
table_name = os.path.basename(path)
table_name = table_name[:table_name.find(".")]
table_name = "%s2_%s2"%(table_name, table_name.lower())
table_name

'Abonnements_Renouvellements_HetPl2_abonnements_renouvellements_hetpl2'

In [47]:
df.shape

(16845, 11)

In [60]:
df_to_sqlite(df, path_sqlite3, table_name, "append")